In [1]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import time
import pandas as pd
import os
from IPython.display import display, HTML
import re

In [2]:
# Fetch BeautifulSoup object
def get_soup(url):
    if not url:
        print("Invalid URL:", url)
        return None
    # Make the request
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup
        else:
            print(f"Failed to fetch the page. Status code: {response.status_code} for URL: {url}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

# Find season from the scorecard page 
def extract_season(url):
    soup = get_soup(url)
    if not soup:
        return None
    
    season = 'N/A'
    
    #Find the row containing the 'Season' span
    rows = soup.find_all('tr')
    for row in rows:
        td = row.find('td',class_= 'ds-min-w-max ds-border-r ds-border-line ds-text-typo-mid1')
        if td and td.find('span', class_= 'ds-text-tight-s ds-font-medium').text.strip() == "Season":
            season_span = row.find('span', class_= 'ds-text-tight-s ds-font-medium ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block')
            if season_span:
                season = season_span.text.strip()
                break
    return season


# Find match date from scorecard page
def extract_match_days(url):
    soup = get_soup(url)
    if not soup:
        return None
    
    match_days = 'N/A'
    
    #Find the row containing the 'Match days' span
    rows = soup.find_all('tr')
    for row in rows:
        td = row.find('td', class_= 'ds-min-w-max ds-border-r ds-border-line ds-text-typo-mid1')
        if td and td.find('span', class_= 'ds-text-tight-s ds-font-medium').text.strip() == "Match days":
            match_days_span = row.find('span', class_= 'ds-text-tight-s ds-font-regular')
            if match_days_span:
                match_days = match_days_span.text.strip()
                break
    return match_days


# Find venue from scorecard page
def extract_venue(url):
    soup = get_soup(url)
    if not soup:
        return None
    
    venue_element = soup.find('span', class_='ds-text-tight-s ds-font-medium ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block') if soup else None
    venue = venue_element.text if venue_element else 'N/A'
    return venue


# Find Teams from scorecard page
def extract_teams(url):
    soup = get_soup(url)
    if not soup:
        return None
    
    teams = 'N/A'
    
    # Finding the team names from the page
    title_tag = soup.find('title')
    if title_tag:
        title_text = title_tag.text
        # Check if "Cricket Scorecard" is in the title text and remove it
        if "Cricket Scorecard" in title_text:
            title_text = title_text.replace("Cricket Scorecard", "").strip()
        # Split the title text to get team names
        if " vs " in title_text:
            team_names = title_text.split(" vs ")
            if len(team_names) >= 2:
                team1 = team_names[0].strip()
                team2 = team_names[1].split(",")[0].strip()  # Remove any trailing text after the team name
                teams = f"{team1} vs {team2}"
                return teams

    return teams


# Find Outcome from scorecard page 
def extract_outcome(url):
    soup = get_soup(url)
    if not soup:
        return None
    
    outcome_element = soup.find('p', class_= 'ds-text-tight-s ds-font-medium ds-truncate ds-text-typo') if soup else None
    outcome = outcome_element.text if outcome_element else 'N/A'
    return outcome
    

# Find Match Situations & Players Involved from match report page
def extract_match_situations_and_players(match_situation_url, player_names):
    soup = get_soup(match_situation_url)
    if not soup:
        return {'Match Situation': 'N/A', 'Players Involved': 'N/A'}
    
    match_situations = 'N/A'
    
     # Find the header tag
    header_tag = soup.find('header')
    if not header_tag:
        return {'Match Situation': 'N/A', 'Players Involved': 'N/A'}
    
    # Find the h1 tag within the header
    h1_tag = header_tag.find('h1')
    h1_text = h1_tag.text.strip() if h1_tag else 'N/A'
    
    # Find the p tag within the header
    p_tag = header_tag.find('p')
    p_text = p_tag.text.strip() if p_tag else 'N/A'
    
    combined_text = f"{h1_text}; {p_text}"


    players_involved = [player for player in player_names if player in combined_text]


    return {
        'Match Situation': combined_text,
        'Players Involved': ', '.join(players_involved) if players_involved else 'N/A'
    }

# Process a single match
def process_match(match):
    print(f"Processing match: {match['scorecard_url']}")
    season = extract_season(match['scorecard_url'])
    match_days = extract_match_days(match['scorecard_url'])
    venue = extract_venue(match['scorecard_url'])
    teams = extract_teams(match['scorecard_url'])
    outcome = extract_outcome(match['scorecard_url'])
    match_situation_and_players = extract_match_situations_and_players(match['match_report_url'], match['player_names'])
    
    result = {
        'Match ID': match['match_id'],
        'Season': season,
        'Match Date': match_days,
        'Venue': venue,
        'Teams': teams,
        'Outcome': outcome,
        'Match Situation': match_situation_and_players['Match Situation'],
        'Players Involved': match_situation_and_players['Players Involved']
    }
    return result

# Batch processing with threading
def process_matches_in_batches(matches, batch_size=5):
    results = []
    for i in range(0, len(matches), batch_size):
        batch = matches[i:i+batch_size]
        with concurrent.futures.ThreadPoolExecutor(max_workers=batch_size) as executor:
            future_to_match = {executor.submit(process_match, match): match for match in batch}
            for future in concurrent.futures.as_completed(future_to_match):
                match = future_to_match[future]
                try:
                    result = future.result()
                    results.append(result)
                except Exception as exc:
                    print(f"Match {match['scorecard_url']} generated an exception: {exc}")
    return results




# List of match URLs 
matches = [
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/ipl-2021-1249214/chennai-super-kings-vs-kolkata-knight-riders-final-1254117/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/ipl-2021-1249214/chennai-super-kings-vs-kolkata-knight-riders-final-1254117/match-report',
        'player_names': ["Faf du Plessis", "Shardul Thakur"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/ipl-2021-1249214/royal-challengers-bangalore-vs-kolkata-knight-riders-eliminator-1254115/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/ipl-2021-1249214/royal-challengers-bangalore-vs-kolkata-knight-riders-eliminator-1254115/match-report',
        'player_names': ["Narine", "Kohli"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/ipl-2021-1249214/royal-challengers-bangalore-vs-mumbai-indians-39th-match-1254108/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/ipl-2021-1249214/royal-challengers-bangalore-vs-mumbai-indians-39th-match-1254108/match-report',
        'player_names': ["Harshal Patel", "Glenn Maxwell", "Yuzvendra Chahal", "Virat Kohli"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/ipl-2021-1249214/delhi-capitals-vs-kolkata-knight-riders-25th-match-1254082/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/ipl-2021-1249214/delhi-capitals-vs-kolkata-knight-riders-25th-match-1254082/match-report',
        'player_names': ["Prithvi Shaw"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2022-1298423/gujarat-titans-vs-rajasthan-royals-final-1312200/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2022-1298423/gujarat-titans-vs-rajasthan-royals-final-1312200/match-report',
        'player_names': ["Hardik Pandya"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2022-1298423/rajasthan-royals-vs-royal-challengers-bangalore-qualifier-2-1312199/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2022-1298423/rajasthan-royals-vs-royal-challengers-bangalore-qualifier-2-1312199/match-report',
        'player_names': ["Buttler", "Prasidh", "McCoy"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2022-1298423/mumbai-indians-vs-delhi-capitals-69th-match-1304115/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2022-1298423/mumbai-indians-vs-delhi-capitals-69th-match-1304115/match-report',
        'player_names': ["Tim David","Jasprit Bumrah","Pant"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2022-1298423/delhi-capitals-vs-sunrisers-hyderabad-50th-match-1304096/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2022-1298423/delhi-capitals-vs-sunrisers-hyderabad-50th-match-1304096/match-report',
        'player_names': ["Warner","Powell"]
    },
    {
    
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2022-1298423/gujarat-titans-vs-mumbai-indians-51st-match-1304097/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2022-1298423/gujarat-titans-vs-mumbai-indians-51st-match-1304097/match-report',
        'player_names': ["Sams","David"]
    },
    {
    
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/gujarat-titans-vs-chennai-super-kings-final-1370353/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/gujarat-titans-vs-chennai-super-kings-final-1370353/match-report',
        'player_names': ["Jadeja"]
    },
    {
    
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/gujarat-titans-vs-mumbai-indians-qualifier-2-1370352/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/gujarat-titans-vs-mumbai-indians-qualifier-2-1370352/match-report',
        'player_names': ["Gill","Mohit"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/mumbai-indians-vs-rajasthan-royals-42nd-match-1359516/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/mumbai-indians-vs-rajasthan-royals-42nd-match-1359516/match-report',
        'player_names': ["David"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/delhi-capitals-vs-chennai-super-kings-67th-match-1359541/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/delhi-capitals-vs-chennai-super-kings-67th-match-1359541/match-report',
        'player_names': ["Gaikwad","Conway"]
    },
    {

        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/sunrisers-hyderabad-vs-lucknow-super-giants-58th-match-1359532/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/sunrisers-hyderabad-vs-lucknow-super-giants-58th-match-1359532/match-report',
        'player_names': ["Mankad","Pooran"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/royal-challengers-bangalore-vs-gujarat-titans-70th-match-1359544/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/royal-challengers-bangalore-vs-gujarat-titans-70th-match-1359544/match-report',
        'player_names': ["Gill"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/mumbai-indians-vs-delhi-capitals-20th-match-1422138/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/mumbai-indians-vs-delhi-capitals-20th-match-1422138/match-report',
        'player_names': ["Shepherd","David","Bumrah","Stubbs","Coetzee"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/sunrisers-hyderabad-vs-royal-challengers-bengaluru-41st-match-1426279/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/sunrisers-hyderabad-vs-royal-challengers-bengaluru-41st-match-1426279/match-report',
        'player_names': ["Patidar","Green","Kohli"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/sunrisers-hyderabad-vs-punjab-kings-69th-match-1426307/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/sunrisers-hyderabad-vs-punjab-kings-69th-match-1426307/match-report',
        'player_names': ["Abhishek"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/rajasthan-royals-vs-mumbai-indians-38th-match-1426276/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/rajasthan-royals-vs-mumbai-indians-38th-match-1426276/match-report',
        'player_names': ["Sandeep Sharma","Yashasvi Jaiswal"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/kolkata-knight-riders-vs-sunrisers-hyderabad-3rd-match-1422121/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/kolkata-knight-riders-vs-sunrisers-hyderabad-3rd-match-1422121/match-report',
        'player_names': ["Russell","Harshit Rana"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/chennai-super-kings-vs-lucknow-super-giants-39th-match-1426277/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/chennai-super-kings-vs-lucknow-super-giants-39th-match-1426277/match-report',
        'player_names': ["Marcus Stoinis"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/punjab-kings-vs-rajasthan-royals-27th-match-1426265/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/punjab-kings-vs-rajasthan-royals-27th-match-1426265/match-report',
        'player_names': ["Hetmyer","Curran"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/royal-challengers-bengaluru-vs-sunrisers-hyderabad-30th-match-1426268/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/royal-challengers-bengaluru-vs-sunrisers-hyderabad-30th-match-1426268/match-report',
        'player_names': ["Head","Klaasen"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/mumbai-indians-vs-chennai-super-kings-29th-match-1426267/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/mumbai-indians-vs-chennai-super-kings-29th-match-1426267/match-report',
        'player_names': ["Pathirana","Rohit"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/royal-challengers-bengaluru-vs-chennai-super-kings-68th-match-1426306/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/royal-challengers-bengaluru-vs-chennai-super-kings-68th-match-1426306/match-report',
        'player_names': ["Yash Dayal"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/lucknow-super-giants-vs-gujarat-titans-21st-match-1422139/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/lucknow-super-giants-vs-gujarat-titans-21st-match-1422139/match-report',
        'player_names': ["Yash Thakur"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/kolkata-knight-riders-vs-rajasthan-royals-31st-match-1426269/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/kolkata-knight-riders-vs-rajasthan-royals-31st-match-1426269/match-report',
        'player_names': ["Buttler"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/mumbai-indians-vs-royal-challengers-bengaluru-25th-match-1426263/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/mumbai-indians-vs-royal-challengers-bengaluru-25th-match-1426263/match-report',
        'player_names': ["Bumrah","Kishan"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/kolkata-knight-riders-vs-delhi-capitals-47th-match-1426285/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/kolkata-knight-riders-vs-delhi-capitals-47th-match-1426285/match-report',
        'player_names': ["Varun","Salt"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/chennai-super-kings-vs-rajasthan-royals-61st-match-1426299/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/chennai-super-kings-vs-rajasthan-royals-61st-match-1426299/match-report',
        'player_names': ["Simarjeet","Gaikwad"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/chennai-super-kings-vs-punjab-kings-41st-match-1359515/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/chennai-super-kings-vs-punjab-kings-41st-match-1359515/match-report',
        'player_names': ["Raza","Pathirana","Devon Conway"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/kolkata-knight-riders-vs-lucknow-super-giants-68th-match-1359542/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/kolkata-knight-riders-vs-lucknow-super-giants-68th-match-1359542/match-report',
        'player_names': ["Pooran","Bishnoi","Rinku Singh"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/sunrisers-hyderabad-vs-royal-challengers-bangalore-65th-match-1359539/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/sunrisers-hyderabad-vs-royal-challengers-bangalore-65th-match-1359539/match-report',
        'player_names': ["Kohli","du Plessis"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/rajasthan-royals-vs-punjab-kings-8th-match-1359482/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/rajasthan-royals-vs-punjab-kings-8th-match-1359482/match-report',
        'player_names': ["Dhawan","Ellis","Prabhsimran","Jurel","Hetmyer"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/rajasthan-royals-vs-chennai-super-kings-37th-match-1359511/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/rajasthan-royals-vs-chennai-super-kings-37th-match-1359511/match-report',
        'player_names': ["Jaiswal","Zampa"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/mumbai-indians-vs-kolkata-knight-riders-22nd-match-1359496/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/mumbai-indians-vs-kolkata-knight-riders-22nd-match-1359496/match-report',
        'player_names': ["Kishan","Suryakumar"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/chennai-super-kings-vs-delhi-capitals-55th-match-1359529/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/chennai-super-kings-vs-delhi-capitals-55th-match-1359529/match-report',
        'player_names': ["Dube","Dhoni"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/sunrisers-hyderabad-vs-delhi-capitals-34th-match-1359508/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/sunrisers-hyderabad-vs-delhi-capitals-34th-match-1359508/match-report',
        'player_names': ["Washington"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/punjab-kings-vs-royal-challengers-bangalore-27th-match-1359501/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/punjab-kings-vs-royal-challengers-bangalore-27th-match-1359501/match-report',
        'player_names': ["Du Plessis","Siraj"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/royal-challengers-bangalore-vs-lucknow-super-giants-15th-match-1359489/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/royal-challengers-bangalore-vs-lucknow-super-giants-15th-match-1359489/match-report',
        'player_names': ["Pooran","Stoinis"]
    },
    {
       
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/kolkata-knight-riders-vs-sunrisers-hyderabad-19th-match-1359493/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/kolkata-knight-riders-vs-sunrisers-hyderabad-19th-match-1359493/match-report',
        'player_names': ["Brook","Nitish Rana","Rinku Singh"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/delhi-capitals-vs-mumbai-indians-16th-match-1359490/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/delhi-capitals-vs-mumbai-indians-16th-match-1359490/match-report',
        'player_names': ["Chawla","Rohit"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/delhi-capitals-vs-kolkata-knight-riders-28th-match-1359502/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/delhi-capitals-vs-kolkata-knight-riders-28th-match-1359502/match-report',
        'player_names': ["Warner"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/rajasthan-royals-vs-lucknow-super-giants-26th-match-1359500/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/rajasthan-royals-vs-lucknow-super-giants-26th-match-1359500/match-report',
        'player_names': ["Avesh","Stoinis"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/mumbai-indians-vs-chennai-super-kings-12th-match-1359486/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/mumbai-indians-vs-chennai-super-kings-12th-match-1359486/match-report',
        'player_names': ["Jadeja","Santner","Rahane"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/royal-challengers-bangalore-vs-kolkata-knight-riders-36th-match-1359510/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2023-1345038/royal-challengers-bangalore-vs-kolkata-knight-riders-36th-match-1359510/match-report',
        'player_names': ["Varun","Suyash"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/gujarat-titans-vs-punjab-kings-17th-match-1422135/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/gujarat-titans-vs-punjab-kings-17th-match-1422135/match-report',
        'player_names': ["Shashank Singh","Ashutosh Sharma"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/delhi-capitals-vs-rajasthan-royals-56th-match-1426294/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/delhi-capitals-vs-rajasthan-royals-56th-match-1426294/match-report',
        'player_names': ["Kuldeep","Mukesh"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/sunrisers-hyderabad-vs-rajasthan-royals-50th-match-1426288/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/sunrisers-hyderabad-vs-rajasthan-royals-50th-match-1426288/match-report',
        'player_names': ["Cummins","Bhuvneshwar"]
    },
    {
        
        'scorecard_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/rajasthan-royals-vs-lucknow-super-giants-4th-match-1422122/full-scorecard',
        'match_report_url': 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/rajasthan-royals-vs-lucknow-super-giants-4th-match-1422122/match-report',
        'player_names': ["Pooran","Rahul"]
    },
]


# Generate unique match IDs for each match in the list
for i, match in enumerate(matches, start=1):
    match['match_id'] = i 

start_time = time.time()
results = process_matches_in_batches(matches, batch_size=5)
end_time = time.time()



Processing match: https://www.espncricinfo.com/series/ipl-2021-1249214/chennai-super-kings-vs-kolkata-knight-riders-final-1254117/full-scorecard
Processing match: https://www.espncricinfo.com/series/ipl-2021-1249214/royal-challengers-bangalore-vs-kolkata-knight-riders-eliminator-1254115/full-scorecard
Processing match: https://www.espncricinfo.com/series/ipl-2021-1249214/royal-challengers-bangalore-vs-mumbai-indians-39th-match-1254108/full-scorecard
Processing match: https://www.espncricinfo.com/series/ipl-2021-1249214/delhi-capitals-vs-kolkata-knight-riders-25th-match-1254082/full-scorecard
Processing match: https://www.espncricinfo.com/series/indian-premier-league-2022-1298423/gujarat-titans-vs-rajasthan-royals-final-1312200/full-scorecard
Processing match: https://www.espncricinfo.com/series/indian-premier-league-2022-1298423/rajasthan-royals-vs-royal-challengers-bangalore-qualifier-2-1312199/full-scorecardProcessing match: https://www.espncricinfo.com/series/indian-premier-league-2

In [3]:
# Convert the list of dictionaries to a DataFrame
match_context_df = pd.DataFrame(results)

# Sort the DataFrame by 'Match ID' column in place
match_context_df.sort_values(by='Match ID', inplace=True)

# Reset the index of the sorted DataFrame in place
match_context_df.reset_index(drop=True, inplace=True)

# Display DataFrame as HTML in Jupyter Notebook
display(HTML(match_context_df.to_html()))

# # Optional: Save to csv
# csv_file = 'critical_moments_in_ipl.csv'
# df.to_csv(csv_file, index = False)

# # Print the location where the file is saved
# print(f"CSV file saved to: {os.path.join(os.getcwd(), csv_file)}")

,Match ID,Season,Match Date,Venue,Teams,Outcome,Match Situation,Players Involved
0,1,2021,15 October 2021 - night (20-over match),Dubai International Cricket Stadium,CSK vs KKR,CSK won by 27 runs,"Faf du Plessis and bowlers stifle KKR to seal CSK's fourth IPL title; Uthappa and Moeen played their parts with the bat too, before Shardul Thakur and Co cut through Knight Riders' middle","Faf du Plessis, Shardul Thakur"
1,2,2021,11 October 2021 - night (20-over match),Sharjah Cricket Stadium,RCB vs KKR,KKR won by 4 wickets (with 2 balls remaining),"Narine stars with bat and ball, puts KKR in Qualifier 2, ends Kohli's tenure as RCB captain with defeat; It was one of the great IPL all-round performances, as Narine's 4 for 21 and 15-ball 26 swung, and changed, the match","Narine, Kohli"
2,3,2021,26 September 2021 - night (20-over match),Dubai International Cricket Stadium,RCB vs MI,RCB won by 54 runs,"Harshal Patel hat-trick caps comprehensive Royal Challengers Bangalore win; Glenn Maxwell, Yuzvendra Chahal and Virat Kohli also fire for RCB; flat Mumbai Indians fall to seventh on points table","Harshal Patel, Glenn Maxwell, Yuzvendra Chahal, Virat Kohli"
3,4,2021,29 April 2021 - night (20-over match),"Narendra Modi Stadium, Ahmedabad",DC vs KKR,DC won by 7 wickets (with 21 balls remaining),"Prithvi Shaw carnage leads Delhi Capitals to comprehensive win against Kolkata Knight Riders; Shaw cracked 82 off just 41 balls, and smashed six fours in the opening over of the chase",Prithvi Shaw
4,5,2022,29 May 2022 - night (20-over match),"Narendra Modi Stadium, Ahmedabad",GT vs RR,GT won by 7 wickets (with 11 balls remaining),All-round Hardik Pandya leads debutants Gujarat Titans to dream title; Captain takes three wickets and scores vital 34 as Titans make light work of Royals' 130,Hardik Pandya
5,6,2022,27 May 2022 - night (20-over match),"Narendra Modi Stadium, Ahmedabad",RR vs RCB,RR won by 7 wickets (with 11 balls remaining),"Buttler ton, Prasidh-McCoy show put Royals in IPL final; Royal Challengers Bangalore were restricted to 157 for 8 and then were beaten with 11 balls to spare","Buttler, Prasidh, McCoy"
6,7,2022,21 May 2022 - night (20-over match),"Wankhede Stadium, Mumbai",MI vs DC,MI won by 5 wickets (with 5 balls remaining),"RCB sneak into playoffs after Tim David, Jasprit Bumrah knock Delhi Capitals out; David would have been out for 0 had Pant opted to review a caught-behind decision. He went onto smash 34 off 11 balls","Tim David, Jasprit Bumrah, Pant"
7,8,2022,5 May 2022 - night (20-over match),"Brabourne Stadium, Mumbai",DC vs SRH,DC won by 21 runs,"Warner, Powell power Capitals convincingly past Sunrisers; Third defeat in a row for Sunrisers, who fall 21 short of target of 208","Warner, Powell"
8,9,2022,6 May 2022 - night (20-over match),"Brabourne Stadium, Mumbai",GT vs MI,MI won by 5 runs,"Sams defends eight in last over after Rohit, David knocks; Gujarat Titans fell short in the last over and lost their second game in a row","Sams, David"
9,10,2023,"28,29 May 2023 - night (20-over match)","Narendra Modi Stadium, Ahmedabad",GT vs CSK,CSK won by 5 wickets (with 0 balls remaining) (DLS method),"Jadeja's last-ball four seals fifth title for CSK in rollercoaster final; After a washed-out day and a long delay, CSK won a thriller off the final ball against Gujarat Titans to spark off celebrations in the early hours of Tuesday",Jadeja


In [4]:
# Dictionary mapping incomplete names to full names
name_mapping = {
    'du Plessis': 'Faf du Plessis', 'Du Plessis': 'Faf du Plessis', 'Narine': 'Sunil Narine','Kohli': 'Virat Kohli','Buttler': 'Jos Buttler','Prasidh': 'Prasidh Krishna','McCoy': 'Obed McCoy',
    'Pant': 'Rishabh Pant','Warner': 'David Warner','Powell': 'Rovman Powell','Sams': 'Daniel Sams','David': 'Tim David','Jadeja': 'Ravindra Jadeja',
    'Gill': 'Shubman Gill','Mohit': 'Mohit Sharma','Gaikwad': 'Ruturaj Gaikwad','Conway': 'Devon Conway','Mankad': 'Prerak Mankad','Pooran': 'Nicholas Pooran',
    'Shepherd': 'Romario Shepherd','Bumrah': 'Jasprit Bumrah','Stubbs': 'Tristan Stubbs','Coetzee': 'Gerald Coetzee','Patidar': 'Rajat Patidar','Green': 'Cameron Green',
    'Abhishek': 'Abhishek Sharma','Russell': 'Andre Russell','Hetmyer': 'Shimron Hetmyer','Curran': 'Sam Curran','Head': 'Travis Head','Klaasen': 'Heinrich Klaasen','Pathirana': 'Matheesha Pathirana',
    'Rohit': 'Rohit Sharma','Kishan': 'Ishan Kishan','Varun': 'Varun Chakaravarthy','Salt': 'Phil Salt','Simarjeet': 'Simarjeet Singh','Raza': 'Sikandar Raza',
    'Bishnoi': 'Ravi Bishnoi','Dhawan': 'Shikhar Dhawan','Ellis': 'Nathan Ellis','Prabhsimran': 'Prabhsimran Singh','Jurel': 'Dhruv Jurel','Jaiswal': 'Yashasvi Jaiswal','Zampa': 'Adam Zampa',
    'Suryakumar': 'Suryakumar Yadav','Dube': 'Shivam Dube','Dhoni': 'MS Dhoni','Washington': 'Washington Sundar','Siraj': 'Mohammed Siraj','Stoinis': 'Marcus Stoinis','Brook': 'Harry Brook',
    'Chawla': 'Piyush Chawla','Avesh': 'Avesh Khan','Santner': 'Mitchell Santner','Rahane': 'Ajinkya Rahane','Suyash': 'Suyash Prabhudessai','Kuldeep': 'Kuldeep Yadav',
    'Mukesh': 'Mukesh Kumar','Cummins': 'Pat Cummins','Bhuvneshwar': 'Bhuvneshwar Kumar','Rahul': 'KL Rahul',    
}


# Split the 'Players Involved' column into multiple rows
expanded_df = match_context_df.set_index(['Match ID', 'Season', 'Match Date', 'Venue', 'Teams', 'Outcome', 'Match Situation'])['Players Involved'].str.split(', ', expand=True).stack().reset_index()

# Rename the columns appropriately
expanded_df = expanded_df.rename(columns={0: 'Player Name'})

# Drop the level_7 column as it's just an artifact from the split operation
expanded_df = expanded_df.drop('level_7', axis=1)

# Replace incomplete names with full names using the mapping dictionary
expanded_df['Player Name'] = expanded_df['Player Name'].replace(name_mapping)

# Extract the 'Player Name' column
player_names = expanded_df['Player Name'].unique()

# Remove duplicates
unique_player_names = expanded_df['Player Name'].unique().tolist()

# # Adjust Pandas display options to show all rows
# pd.set_option('display.max_rows', None)

# Create a DataFrame with a column named "Player Name"
player_names_df = pd.DataFrame({'Player Name': unique_player_names})

# # Display DataFrame
# print(player_names_df)



In [5]:
# Dictionary mapping player names to their respective teams
player_team_mapping = {
    'Faf du Plessis': 'RCB','Shardul Thakur': 'CSK','Sunil Narine': 'KKR','Virat Kohli': 'RCB','Harshal Patel': 'PBKS','Glenn Maxwell': 'RCB','Yuzvendra Chahal': 'RR',
    'Prithvi Shaw': 'DC','Hardik Pandya': 'MI','Jos Buttler': 'RR','Prasidh Krishna': 'RR*','Obed McCoy': 'RR','Tim David': 'MI','Jasprit Bumrah': 'MI',
    'Rishabh Pant': 'DC','David Warner': 'DC','Rovman Powell': 'DC','Daniel Sams': 'MI','Ravindra Jadeja': 'CSK','Shubman Gill': 'GT','Mohit Sharma': 'GT',
    'Ruturaj Gaikwad': 'CSK','Devon Conway': 'CSK','Prerak Mankad': 'LSG','Nicholas Pooran': 'LSG','Romario Shepherd': 'MI','Tristan Stubbs': 'DC','Gerald Coetzee': 'MI',
    'Rajat Patidar': 'RCB','Cameron Green': 'RCB','Abhishek Sharma': 'SRH','Sandeep Sharma': 'RR','Yashasvi Jaiswal': 'RR','Andre Russell': 'KKR','Harshit Rana': 'KKR','Marcus Stoinis': 'LSG',
    'Shimron Hetmyer': 'RR','Sam Curran': 'PBKS','Travis Head': 'SRH','Heinrich Klaasen': 'SRH','Matheesha Pathirana': 'CSK','Rohit Sharma': 'MI','Yash Dayal': 'RCB',
    'Yash Thakur': 'LSG','Ishan Kishan': 'MI','Varun Chakaravarthy': 'KKR','Phil Salt': 'KKR','Simarjeet Singh': 'CSK','Sikandar Raza': 'PBKS','Ravi Bishnoi': 'LSG',
    'Rinku Singh': 'KKR','Shikhar Dhawan': 'PBKS','Nathan Ellis': 'PBKS','Prabhsimran Singh': 'PBKS','Dhruv Jurel': 'RR','Adam Zampa': 'RR','Suryakumar Yadav': 'MI','Shivam Dube': 'CSK',
    'MS Dhoni': 'CSK','Washington Sundar': 'SRH','Mohammed Siraj': 'RCB','Harry Brook': 'SRH','Nitish Rana': 'KKR','Piyush Chawla': 'MI','Avesh Khan': 'RR','Mitchell Santner': 'CSK',
    'Ajinkya Rahane': 'CSK','Suyash Prabhudessai': 'RCB','Shashank Singh': 'PBKS','Ashutosh Sharma': 'PBKS','Kuldeep Yadav': 'DC','Mukesh Kumar': 'DC','Pat Cummins': 'SRH',
    'Bhuvneshwar Kumar': 'SRH','KL Rahul': 'LSG',
}

# Create a new column 'Team' by mapping player names to their respective teams in expanded_df
expanded_df['Team'] = expanded_df['Player Name'].map(player_team_mapping)

# # Display the updated DataFrame
# print(player_names_df)
# print("* Former Team Player")

In [6]:
# Add a new column 'Contribution Description' and set it to None initially
expanded_df['Contribution Description', 'Impact Rating'] = None

# Manually assign contribution descriptions for specific rows
expanded_df.loc[(expanded_df['Player Name'] == 'Faf du Plessis') & (expanded_df['Match ID'] == 1), ['Contribution Description','Impact Rating']] = ["Faf du Plessis led the charge, making 86 as he anchored half-century stands for each of the three wickets that fell during the course of Super Kings' 192.",81.91]
expanded_df.loc[(expanded_df['Player Name'] == 'Shardul Thakur') & (expanded_df['Match ID'] == 1), ['Contribution Description','Impact Rating']] = ["Shardul Thakur took the most wickets along with other bowlers which led to their victory.",57.8]

expanded_df.loc[(expanded_df['Player Name'] == 'Sunil Narine') & (expanded_df['Match ID'] == 2), ['Contribution Description','Impact Rating']] = ["Three balls of Sunil Narine, with the bat, changed the game, putting Kolkata Knight Riders one step closer to the IPL final.",158.09]
expanded_df.loc[(expanded_df['Player Name'] == 'Virat Kohli') & (expanded_df['Match ID'] == 2), ['Contribution Description','Impact Rating']] = ["Pressure was mounting on Kohli. He had slowed down once again after the powerplay - 24 off 16 and 15 off 17 - and his desperation was apparent.",46.38]

expanded_df.loc[(expanded_df['Player Name'] == 'Virat Kohli') & (expanded_df['Match ID'] == 3), ['Contribution Description','Impact Rating']] = ["Virat Kohli struck another half-century",62.36]
expanded_df.loc[(expanded_df['Player Name'] == 'Harshal Patel') & (expanded_df['Match ID'] == 3), ['Contribution Description','Impact Rating']] =[ "Glenn Maxwell provided middle-overs firepower and key wickets",34.83]
expanded_df.loc[(expanded_df['Player Name'] == 'Glenn Maxwell') & (expanded_df['Match ID'] == 3), ['Contribution Description','Impact Rating']] = ["Chahal took 3 wickets giving as less as 11 runs",137.88]
expanded_df.loc[(expanded_df['Player Name'] == 'Yuzvendra Chahal') & (expanded_df['Match ID'] == 3), ['Contribution Description','Impact Rating']] =[ "Patel bagged a hat-trick key wicket strike",74.45]

expanded_df.loc[(expanded_df['Player Name'] == 'Prithvi Shaw') & (expanded_df['Match ID'] == 4), ['Contribution Description','Impact Rating']] = ["Shaw cracked 82 off just 41 balls, and smashed six fours in the opening over of the chase",112.14]

expanded_df.loc[(expanded_df['Player Name'] == 'Hardik Pandya') & (expanded_df['Match ID'] == 5), ['Contribution Description','Impact Rating']] = ["Captain Hardik Pandya starred with the ball and bat to lead Gujarat Titans to IPL glory",137.15]

expanded_df.loc[(expanded_df['Player Name'] == 'Jos Buttler') & (expanded_df['Match ID'] == 6), ['Contribution Description','Impact Rating']] = ["The unstoppable Buttler scores a century very easily",137.47]
expanded_df.loc[(expanded_df['Player Name'] == 'Prasidh Krishna') & (expanded_df['Match ID'] == 6), ['Contribution Description','Impact Rating']] = ["Prasidh took 3 wickets, leading to victory.",88.02]
expanded_df.loc[(expanded_df['Player Name'] == 'Obed McCoy') & (expanded_df['Match ID'] == 6), ['Contribution Description','Impact Rating']] = ["McCoy took 3 wickets, leading to victory.",72.55]

expanded_df.loc[(expanded_df['Player Name'] == 'Tim David') & (expanded_df['Match ID'] == 7), ['Contribution Description','Impact Rating']] = ["David kept hitting sixes. He got to face 10 more balls and score 34 heartbreaking runs",73.12]
expanded_df.loc[(expanded_df['Player Name'] == 'Jasprit Bumrah') & (expanded_df['Match ID'] == 7), ['Contribution Description','Impact Rating']] = ["Jasprit Bumrah (4-0-25-3) was extraordinary. Even on a slow pitch, he hurried batters with his pace and picked up the kind of wickets that will form a fast-bowling hype reel.",81.36]
expanded_df.loc[(expanded_df['Player Name'] == 'Rishabh Pant ') & (expanded_df['Match ID'] == 7), ['Contribution Description','Impact Rating']] = ["Pant tried his best as batsman scoring as much as 39 runs in 33 balls.",37.02]

expanded_df.loc[(expanded_df['Player Name'] == 'David Warner') & (expanded_df['Match ID'] == 8), ['Contribution Description','Impact Rating']] = ["David Warner took centre stage, an unbeaten 92 from 58. Key patnership with Powell.",95.39]
expanded_df.loc[(expanded_df['Player Name'] == 'Rovman Powell') & (expanded_df['Match ID'] == 8), ['Contribution Description','Impact Rating']] = ["Powell finished with three fours and six sixes in 67 from 35, with 70 runs cascading from the last five overs of the innings to put the chase beyond Sunrisers.",75.05]

expanded_df.loc[(expanded_df['Player Name'] == 'Tim David') & (expanded_df['Match ID'] == 9), ['Contribution Description','Impact Rating']] = ["David, who took them to 177 with two sixes in the final over, was despondent at the halfway mark",66.45]
expanded_df.loc[(expanded_df['Player Name'] == 'Daniel Sams') & (expanded_df['Match ID'] == 9), ['Contribution Description','Impact Rating']] = ["Sams managed to go past the bat on both occasions with full and wide slower balls.",15.16]

expanded_df.loc[(expanded_df['Player Name'] == 'Ravindra Jadeja') & (expanded_df['Match ID'] == 10), ['Contribution Description','Impact Rating']] = ["Jadejas last-ball four seals fifth title for CSK in rollercoaster final",68.81]

expanded_df.loc[(expanded_df['Player Name'] == 'Shubman Gill') & (expanded_df['Match ID'] == 11), ['Contribution Description','Impact Rating']] = ["Shubman Gills majestic 129 off 60 balls powered Gujarat Titans into the final of IPL 2023",164.45]
expanded_df.loc[(expanded_df['Player Name'] == 'Mohit Sharma') & (expanded_df['Match ID'] == 11), ['Contribution Description','Impact Rating']] = ["Mohit, who hadnt bowled till the 14th over, used his slower balls to good effect and finished with figures of 2.2-0-10-5.",37.39]

expanded_df.loc[(expanded_df['Player Name'] == 'Tim David') & (expanded_df['Match ID'] == 12), ['Contribution Description','Impact Rating']] = ["David goes 6, 6, 6 in final over creating a twist",75.65]

expanded_df.loc[(expanded_df['Player Name'] == 'Ruturaj Gaikwad') & (expanded_df['Match ID'] == 13), ['Contribution Description','Impact Rating']] = ["Gaikwad scored 79, and caught 2 catches.",89.22]
expanded_df.loc[(expanded_df['Player Name'] == 'Devon Conway') & (expanded_df['Match ID'] == 13), ['Contribution Description','Impact Rating']] = ["Conway scored 87, having a great partnership with Gaikwad.",94.45]

expanded_df.loc[(expanded_df['Player Name'] == 'Prerak Mankad') & (expanded_df['Match ID'] == 14), ['Contribution Description','Impact Rating']] = ["Mankad 64, led him to be the player of the match",57.3]
expanded_df.loc[(expanded_df['Player Name'] == 'Nicholas Pooran') & (expanded_df['Match ID'] == 14), ['Contribution Description','Impact Rating']] =[ "Pooran stayed unbeaten on 44 off 13.",77.25]

expanded_df.loc[(expanded_df['Player Name'] == 'Shubman Gill') & (expanded_df['Match ID'] == 15), ['Contribution Description','Impact Rating']] = ["Gills unbeaten 104 featured seven more sixes than Kohlis 101 not out",141.54]

expanded_df.loc[(expanded_df['Player Name'] == 'Romario Shepherd') & (expanded_df['Match ID'] == 16), ['Contribution Description','Impact Rating']] = ["Romario Shepherd (39* off 10), Shepherd who made the bigger dent. In the 20th over, he bludgeoned Nortje for 4, 6, 6, 6, 4, 6 to crush Capitals morale.",93.01]
expanded_df.loc[(expanded_df['Player Name'] == 'Tristan Stubbs') & (expanded_df['Match ID'] == 16), ['Contribution Description','Impact Rating']] = ["Stubbs hit two sixes off Shepherd at the start of the 19th and ended it with another six",71.28]
expanded_df.loc[(expanded_df['Player Name'] == 'Gerald Coetzee') & (expanded_df['Match ID'] == 16), ['Contribution Description','Impact Rating']] = ["Coetzee conceded only four and also picked up three wickets as Stubbs got stuck at the non-striker's end.",27.21]
expanded_df.loc[(expanded_df['Player Name'] == 'Jasprit Bumrah') & (expanded_df['Match ID'] == 16), ['Contribution Description','Impact Rating']] = ["Bumrah conceded only eight in the next to make it 55 needed from two overs.",46.29]
expanded_df.loc[(expanded_df['Player Name'] == 'Tim David') & (expanded_df['Match ID'] == 16), ['Contribution Description','Impact Rating']] = ["Tim David (45* off 21), David started the 16th over by smashing Richardson for a six. In the next two overs, he hit two more sixes",52.99]

expanded_df.loc[(expanded_df['Player Name'] == 'Rajat Patidar') & (expanded_df['Match ID'] == 17), ['Contribution Description','Impact Rating']] = ["Rajat Patidars 19-ball half-century",85.47]
expanded_df.loc[(expanded_df['Player Name'] == 'Cameron Green') & (expanded_df['Match ID'] == 17), ['Contribution Description','Impact Rating']] = ["Green ensured RCB got the strong finish they needed taking 2 wickets.",55.2]
expanded_df.loc[(expanded_df['Player Name'] == 'Virat Kohli') & (expanded_df['Match ID'] == 17), ['Contribution Description','Impact Rating']] = ["Virat Kohli also scored his 53rd IPL fifty, but his 51 in 43 balls came at a strike rate of 118.60, following a big slowdown.",30.11]

expanded_df.loc[(expanded_df['Player Name'] == 'Abhishek Sharma') & (expanded_df['Match ID'] == 18), ['Contribution Description','Impact Rating']] = ["Abhishek gave the SRH faithful a show to remember a batting-powered season by, and went on to hit six sixes in all, in addition to five fours.",113.54]

expanded_df.loc[(expanded_df['Player Name'] == 'Sandeep Sharma') & (expanded_df['Match ID'] == 19), ['Contribution Description','Impact Rating']] = ["Sandeep Sharma magnificent figures of 5 for 18, the best by any bowler in this seasons IPL",148.93]
expanded_df.loc[(expanded_df['Player Name'] == 'Yashasvi Jaiswal') & (expanded_df['Match ID'] == 19), ['Contribution Description','Impact Rating']] =[ "Yashasvi Jaiswal blazed back to form with a sparkling 59-ball century",132.11]

expanded_df.loc[(expanded_df['Player Name'] == 'Andre Russell') & (expanded_df['Match ID'] == 20), ['Contribution Description','Impact Rating']] = ["Russell put the finishing touches to the KKR innings by smacking seven sixes in his 25-ball, unbeaten 64.",165.1]
expanded_df.loc[(expanded_df['Player Name'] == 'Harshit Rana') & (expanded_df['Match ID'] == 20), ['Contribution Description','Impact Rating']] = ["Harshit Rana successfully defended 12 off the final over to take KKR over the line in a last-ball thriller.",104.71]

expanded_df.loc[(expanded_df['Player Name'] == 'Marcus Stoinis') & (expanded_df['Match ID'] == 21), ['Contribution Description','Impact Rating']] = ["The star of the night was Marcus Stoinis, whose unbeaten 124 off 63 balls helped LSG pull off the highest successful chase at Chepauk in T20 cricket.",165.39]

expanded_df.loc[(expanded_df['Player Name'] == 'Shimron Hetmyer') & (expanded_df['Match ID'] == 22), ['Contribution Description','Impact Rating']] = ["Shimron Hetmyer came in with Rajasthan Royals needing 35 from 20. He scored 27 off 10 balls.",55.35]
expanded_df.loc[(expanded_df['Player Name'] == 'Sam Curran') & (expanded_df['Match ID'] == 22), ['Contribution Description','Impact Rating']] = ["Despite an excellent 19th over from Curran and a good effort from Arshdeep. Sam took 2 key wickets.",52.48]

expanded_df.loc[(expanded_df['Player Name'] == 'Travis Head') & (expanded_df['Match ID'] == 23), ['Contribution Description','Impact Rating']] = ["Head scored a 24-ball 62, belted a career-best 102 off 41",103.35]
expanded_df.loc[(expanded_df['Player Name'] == 'Heinrich Klaasen') & (expanded_df['Match ID'] == 23), ['Contribution Description','Impact Rating']] = ["Head hit eight sixes, and Heinrich Klaasen seven in a 31-ball 67, out of a Sunrisers total of 22 - another IPL record.",51.71]

expanded_df.loc[(expanded_df['Player Name'] == 'Matheesha Pathirana') & (expanded_df['Match ID'] == 24), ['Contribution Description','Impact Rating']] = ["Pathirana took 4 wickets",114.7]
expanded_df.loc[(expanded_df['Player Name'] == 'Rohit Sharma') & (expanded_df['Match ID'] == 24), ['Contribution Description','Impact Rating']] = ["Rohit Sharma scored a century",96.93]

expanded_df.loc[(expanded_df['Player Name'] == 'Yash Dayal') & (expanded_df['Match ID'] == 25), ['Contribution Description','Impact Rating']] = ["Yash Dayal held his nerve and gave away just seven runs in the final over to help RCB seal a thriller and took 4 wickets.",29.13]

expanded_df.loc[(expanded_df['Player Name'] == 'Yash Thakur') & (expanded_df['Match ID'] == 26), ['Contribution Description','Impact Rating']] = ["Yash Thakur claimed a maiden five-for to help demolish the innings.",64.53]

expanded_df.loc[(expanded_df['Player Name'] == 'Jos Buttler') & (expanded_df['Match ID'] == 27), ['Contribution Description','Impact Rating']] = ["Buttlers 107* tops Narines 109 as Royals ace record chase against KKR",145.12]

expanded_df.loc[(expanded_df['Player Name'] == 'Jasprit Bumrah') & (expanded_df['Match ID'] == 28), ['Contribution Description','Impact Rating']] = ["Bumrahs 5 for 21 - he was on a hat-trick twice",162.3]
expanded_df.loc[(expanded_df['Player Name'] == 'Ishan Kishan') & (expanded_df['Match ID'] == 28), ['Contribution Description','Impact Rating']] = ["Ishan Kishan cracked a 23-ball half-century inside the powerplay in the chase",83.21]

expanded_df.loc[(expanded_df['Player Name'] == 'Varun Chakaravarthy') & (expanded_df['Match ID'] == 29), ['Contribution Description','Impact Rating']] = ["Varun took 3 wickets",83.94]
expanded_df.loc[(expanded_df['Player Name'] == 'Phil Salt') & (expanded_df['Match ID'] == 29), ['Contribution Description','Impact Rating']] = ["Phil Salt made his fourth fifty in five home games this season, dominating a powerplay in which KKR rushed to 79 for no loss",81.99]

expanded_df.loc[(expanded_df['Player Name'] == 'Simarjeet Singh') & (expanded_df['Match ID'] == 30), ['Contribution Description','Impact Rating']] = ["Simarjeet Singh kept getting wickets every time Royals tried to push back.",95.88]
expanded_df.loc[(expanded_df['Player Name'] == 'Ruturaj Gaikwad') & (expanded_df['Match ID'] == 30), ['Contribution Description','Impact Rating']] = ["Captain Gaikwad stayed true to his anchor role for this chase, and saw his team home despite Jadeja getting himself out obstructing the field",28.64]

expanded_df.loc[(expanded_df['Player Name'] == 'Sikandar Raza') & (expanded_df['Match ID'] == 31), ['Contribution Description','Impact Rating']] = ["Even though Pathirana sent down an excellent last over, Raza manipulated the field perfectly to pick ones and twos and sealed the match with a hard-run three.",36.8]
expanded_df.loc[(expanded_df['Player Name'] == 'Matheesha Pathirana') & (expanded_df['Match ID'] == 31), ['Contribution Description','Impact Rating']] = ["With nine to win off the last over, Raza and Shahrukh Khan couldnt hit a single boundary against Pathiranas dipping deliveries.",48.67]
expanded_df.loc[(expanded_df['Player Name'] == 'Devon Conway') & (expanded_df['Match ID'] == 31), ['Contribution Description','Impact Rating']] = ["Devon Conway walloped an unbeaten 92,",105.83]

expanded_df.loc[(expanded_df['Player Name'] == 'Nicholas Pooran') & (expanded_df['Match ID'] == 32), ['Contribution Description','Impact Rating']] = ["At 73 for 5 in the 11th over, they didnt look like scoring even that many, but Nicholas Poorans 30-ball 58 lifted them.",97.04]
expanded_df.loc[(expanded_df['Player Name'] == 'Ravi Bishnoi') & (expanded_df['Match ID'] == 32), ['Contribution Description','Impact Rating']] = ["Bishnoi took 2 crucial wickets",78.06]
expanded_df.loc[(expanded_df['Player Name'] == 'Rinku Singh') & (expanded_df['Match ID'] == 32), ['Contribution Description','Impact Rating']] = ["En route to his unbeaten 67 off 33 balls, he took Naveen for three fours and six in the 19th over but with 18 needed from three balls, he could hit only 6, 4 and 6.",97.92]

expanded_df.loc[(expanded_df['Player Name'] == 'Faf du Plessis') & (expanded_df['Match ID'] == 33), ['Contribution Description','Impact Rating']] = ["Du Plessis had key partnership with Kohli, he scored 71",71.8]
expanded_df.loc[(expanded_df['Player Name'] == 'Virat Kohli') & (expanded_df['Match ID'] == 33), ['Contribution Description','Impact Rating']] = ["Kohli scored a century",99.06]

expanded_df.loc[(expanded_df['Player Name'] == 'Shikhar Dhawan') & (expanded_df['Match ID'] == 34), ['Contribution Description','Impact Rating']] = ["Dhawan anchored a score with an unbeaten 86 off 56",76.03]
expanded_df.loc[(expanded_df['Player Name'] == 'Nathan Ellis') & (expanded_df['Match ID'] == 34), ['Contribution Description','Impact Rating']] = ["Nathan Ellis four-wicket haul was vital in hobbling Royals in the chase",122.68]
expanded_df.loc[(expanded_df['Player Name'] == 'Prabhsimran Singh') & (expanded_df['Match ID'] == 34), ['Contribution Description','Impact Rating']] = ["Prabhsimran easily scored a half century",81.38]
expanded_df.loc[(expanded_df['Player Name'] == 'Dhruv Jurel') & (expanded_df['Match ID'] == 34), ['Contribution Description','Impact Rating']] = ["Batting at No. 8, the 22-year-old showed both power and a cool temperament to thump 32 from 15 in only his fourth T20 innings, as Royals seventh-wicket pair took the game to the wire.",41.24]
expanded_df.loc[(expanded_df['Player Name'] == 'Shimron Hetmyer') & (expanded_df['Match ID'] == 34), ['Contribution Description','Impact Rating']] = ["Key partnership with Jurel.",49.1]

expanded_df.loc[(expanded_df['Player Name'] == 'Yashasvi Jaiswal') & (expanded_df['Match ID'] == 35), ['Contribution Description','Impact Rating']] = ["Yashasvi Jaiswal attacked his way to 77 off 43 balls",90.94]
expanded_df.loc[(expanded_df['Player Name'] == 'Adam Zampa') & (expanded_df['Match ID'] == 35), ['Contribution Description','Impact Rating']] = ["the legspinner made an impact in his very first over. Bowling the last over of the powerplay, he had Devon Conway hitting one to mid-off. Zampa took 3 wickets",57.09]

expanded_df.loc[(expanded_df['Player Name'] == 'Ishan Kishan') & (expanded_df['Match ID'] == 36), ['Contribution Description','Impact Rating']] = ["When Narine came in to bowl the chases fourth over, he was hit for 22 with Kishan going 6, dot, 6, 4 through slogs. He got his fifty in 21 balls",104.32]
expanded_df.loc[(expanded_df['Player Name'] == 'Suryakumar Yadav') & (expanded_df['Match ID'] == 36), ['Contribution Description','Impact Rating']] = ["When Lockie Ferguson took the ball, Suryakumar brought out some vintage shots, picking up a full ball for six over fine leg and repeating it next ball over midwicket. Tilak and Suryakumar then picked off Russell for 17 in one over.",46.69]

expanded_df.loc[(expanded_df['Player Name'] == 'Shivam Dube') & (expanded_df['Match ID'] == 37), ['Contribution Description','Impact Rating']] = ["Even though Dube fell for 25, his runs at a strike-rate of 208.33 served as a reminder that poor deliveries still deserved to be dispatched. When he fell, CSK were 113 for 5 in 14.2 overs.",48.59]
expanded_df.loc[(expanded_df['Player Name'] == 'MS Dhoni') & (expanded_df['Match ID'] == 37), ['Contribution Description','Impact Rating']] = ["MS Dhoni who made 20 in nine deliveries, Dhonis late hits lifted CSK to 167",38.57]

expanded_df.loc[(expanded_df['Player Name'] == 'Washington Sundar') & (expanded_df['Match ID'] == 38), ['Contribution Description','Impact Rating']] = ["For a large part of the contest, Sunrisers Hyderabad seemed to have a hold on Delhi Capitals. Like when Washington Sundar picked up three wickets in an over.",91.01]

expanded_df.loc[(expanded_df['Player Name'] == 'Faf du Plessis') & (expanded_df['Match ID'] == 39), ['Contribution Description','Impact Rating']] = ["He hit 84 off 56 balls on a sluggish Mohali pitch to haul Royal Challengers Bangalore to 174 for 4, despite a late slowdown",102.05]
expanded_df.loc[(expanded_df['Player Name'] == 'Mohammed Siraj') & (expanded_df['Match ID'] == 39), ['Contribution Description','Impact Rating']] = ["Siraj took 4 key wickets which made his powerplay tally this season up to six",116.03]

expanded_df.loc[(expanded_df['Player Name'] == 'Nicholas Pooran') & (expanded_df['Match ID'] == 40), ['Contribution Description','Impact Rating']] = ["He ended with 62 from 19. Seven of those balls were hit for sixes, and four of them for fours. Carefree, unstoppable T20 batting, with the swag, muscle and calmness of Sir Viv",134.24]
expanded_df.loc[(expanded_df['Player Name'] == 'Marcus Stoinis') & (expanded_df['Match ID'] == 40), ['Contribution Description','Impact Rating']] = ["After seven overs, Super Giants required rate had crossed 13, but Stoinis welcomed Harshal with 6, 4, 4. Next over, Karn Sharma was given the same treatment.Stoinis departed for 65 from 30 deliveries",112.04]

expanded_df.loc[(expanded_df['Player Name'] == 'Harry Brook') & (expanded_df['Match ID'] == 41), ['Contribution Description','Impact Rating']] = ["Harry Brook put behind a string of low scores with an unbeaten 55-ball 100 that included imperious hitting against pace and calculated strike rotation against spin.",98.68]
expanded_df.loc[(expanded_df['Player Name'] == 'Nitish Rana') & (expanded_df['Match ID'] == 41), ['Contribution Description','Impact Rating']] = ["Nitish Ranas 41-ball 75 kept their hopes alive, and with 58 required from the last three overs",83.76]
expanded_df.loc[(expanded_df['Player Name'] == 'Rinku Singh') & (expanded_df['Match ID'] == 41), ['Contribution Description','Impact Rating']] = ["Rinku fought hard with an unbeaten 58 off 31 but this time, it wasnt to be.",43.49]

expanded_df.loc[(expanded_df['Player Name'] == 'Rohit Sharma') & (expanded_df['Match ID'] == 42), ['Contribution Description','Impact Rating']] = ["The early exchanges of the run-chase had fallen in the visitors favour too, with Rohits 65 from 45 including a 29-ball fifty that provided the impetus for a 68-run powerplay.",74.27]
expanded_df.loc[(expanded_df['Player Name'] == 'Piyush Chawla') & (expanded_df['Match ID'] == 42), ['Contribution Description','Impact Rating']] = ["In an unbroken spell from the seventh to the 13th overs, Chawla returned the exceptional figures of 3 for 22, with only a brace of Warner boundaries disrupting his otherwise complete hold over Capitals batters.",92.93]

expanded_df.loc[(expanded_df['Player Name'] == 'David Warner') & (expanded_df['Match ID'] == 43), ['Contribution Description','Impact Rating']] = ["Warner stood at one end while wickets tumbled from the other. But Warner motored along to get his fourth fifty in six games this season, this one his quickest in 33 balls. The small target allowed him to keep playing his shots, and he had hit 11 fours by the time he raised his bat for the half-century.",83.53]

expanded_df.loc[(expanded_df['Player Name'] == 'Marcus Stoinis') & (expanded_df['Match ID'] == 44), ['Contribution Description','Impact Rating']] = ["Royals were moving towards their target with little fuss when Stoinis ended the opening stand with Jaiswals wicket, who steered one low to Avesh at short third. The next two overs brought two more wickets",82.46]
expanded_df.loc[(expanded_df['Player Name'] == 'Avesh Khan') & (expanded_df['Match ID'] == 44), ['Contribution Description','Impact Rating']] = ["Hetmyer tried to take on Avesh but was caught at long-on, only for 2. That turned the game in Super Giants favour, and from there on, they didnt give Royals any chance.",82.57]

expanded_df.loc[(expanded_df['Player Name'] == 'Ravindra Jadeja') & (expanded_df['Match ID'] == 45), ['Contribution Description','Impact Rating']] = ["Dhoni brought Jadeja into the attack immediately after the powerplay and the allrounder responded by having Kishan hole out for 32 off 21 balls.Jadeja kept bowling into the Wankhede pitch and came away with the wickets of Tilak Varma (22) and Cameron Green (12) as well.",84.77]
expanded_df.loc[(expanded_df['Player Name'] == 'Mitchell Santner') & (expanded_df['Match ID'] == 45), ['Contribution Description','Impact Rating']] = ["Santner might not have played this game had Stokes or Moeen been available for selection, but he proved his worth once again. In stark contrast to Jadeja, Santner tossed the ball up at a much slower pace and dismissed both Suryakumar Yadav (1) and the left-handed Arshad Khan",54.06]
expanded_df.loc[(expanded_df['Player Name'] == 'Ajinkya Rahane') & (expanded_df['Match ID'] == 45), ['Contribution Description','Impact Rating']] = [" Ajinkya Rahane, another Mumbai boy, dashed out of the blocks for Super Kings, hitting a 19-ball half-century to set up their chase. His 27-ball 61 on Super Kings debut allowed Ruturaj Gaikwad and rest of the batters breathing room after Devon Conway had fallen for a duck.",109.2]

expanded_df.loc[(expanded_df['Player Name'] == 'Suyash Prabhudessai') & (expanded_df['Match ID'] == 46), ['Contribution Description','Impact Rating']] = ["Suyash being the least experienced spinner led to wreck of RCB",3.06]
expanded_df.loc[(expanded_df['Player Name'] == 'Varun Chakaravarthy') & (expanded_df['Match ID'] == 46), ['Contribution Description','Impact Rating']] = ["Varun took 3 wickets",72.84]

expanded_df.loc[(expanded_df['Player Name'] == 'Shashank Singh') & (expanded_df['Match ID'] == 47), ['Contribution Description','Impact Rating']] = ["Shashank Singh hit 10 of the 29 balls he faced to the boundary. But the match got twisted when Shashank played unbelievable shots, scoring 61 runs",112.17]
expanded_df.loc[(expanded_df['Player Name'] == 'Ashutosh Sharma') & (expanded_df['Match ID'] == 47), ['Contribution Description','Impact Rating']] = ["Ashutosh Sharma, playing his very first IPL game, and only the 15th T20 of his career, looked at an equation that read 41 off 18 balls square in the eye and took it down with brutal precision. He hit three fours in Azmatullah Omarzais 18th over. Another six to start the 19th from Mohit. In those 12 balls, the Kings got 34 runs. In those 12 balls, the Kings chances of victory rose from 9.23% to 94.56%.",36.99]

expanded_df.loc[(expanded_df['Player Name'] == 'Kuldeep Yadav') & (expanded_df['Match ID'] == 48), ['Contribution Description','Impact Rating']] = ["Kuldeeps double-wicket 18th over gave a knockout blow.",83.29]
expanded_df.loc[(expanded_df['Player Name'] == 'Mukesh Kumar') & (expanded_df['Match ID'] == 48), ['Contribution Description','Impact Rating']] = ["Mukesh also took 2 key wickets",44.22] 

expanded_df.loc[(expanded_df['Player Name'] == 'Pat Cummins') & (expanded_df['Match ID'] == 49), ['Contribution Description','Impact Rating']] = ["Pat Cummins bowled a superb 19th over",75.42]
expanded_df.loc[(expanded_df['Player Name'] == 'Bhuvneshwar Kumar') & (expanded_df['Match ID'] == 49), ['Contribution Description','Impact Rating']] = ["Bhuvneshwar got the new ball moving, drawing the outside edge from Buttler first ball and going past Samsons inside edge in the same over. These were his 45th and 46th wickets in the first over in T20 cricket",75.82]

expanded_df.loc[(expanded_df['Player Name'] == 'Nicholas Pooran') & (expanded_df['Match ID'] == 50), ['Contribution Description','Impact Rating']] = ["Key patnership with KL Rahul",64.06]
expanded_df.loc[(expanded_df['Player Name'] == 'KL Rahul') & (expanded_df['Match ID'] == 50), ['Contribution Description','Impact Rating']] = ["KL Rahul - on his return from injury - teamed up with Nicholas Pooran to take the chase deep in a fifth-wicket stand of 85 from 52 balls that ended from the first ball of the 17th over",52.86]

# # Drop the column
expanded_df.drop(columns=[('Contribution Description', 'Impact Rating')], inplace=True)

# # Display the updated DataFrame
# print(expanded_df)

In [7]:
# Define regular expressions for batting and bowling performance
batting_keywords = ['half-century','making','bat','powerplay','struck','provided','middle-overs','smash','sixes','six',
                   'fours','chase','century','score','hitting','scoring','batsman','unbeaten','partnership','runs','over',
                   'balls','ball','off','scored','player','dent','hit','smashing','fifty','strike','form','smacking','run',
                   'batting','thump','innings','attacked','shots','made','deliveries','from','dashed','faced','key','teamed']

bowling_keywords = ['wickets','took','bagged','fast-bowling','over','finished','picked','conceded','taking','figures',
                   'bowler','defended','key','gave','maiden','for','getting','dipping','crucial','haul','wicket','leg spinner',
                   'bowling','ended','caught','chance','tossed','left-handed','spinner','bowled','ball']

# all_rounder_keywords = ['bat','ball','caught','catches']

# Function to classify key role based on contribution description
def classify_key_role(description):
    if not isinstance(description, str):
        return 'Unknown'
    description_lower = description.lower()
    if any(re.search(r'\b{}\b'.format(keyword), description_lower) for keyword in batting_keywords):
        return 'Batsman'
    elif any(re.search(r'\b{}\b'.format(keyword), description_lower) for keyword in bowling_keywords):
        return 'Bowler'
    elif any(re.search(r'\b{}\b'.format(keyword), description_lower) for keyword in all_rounder_keywords):
        return 'All-rounder'
    else:
        return 'Unknown'  # Default to unknown if no clear indication

# Apply the function to create the 'keyrole' column
expanded_df['keyrole'] = expanded_df['Contribution Description'].apply(classify_key_role)
# Drop the existing 'Key Role' column if it exists
expanded_df.drop(columns=['Key Role'], errors='ignore', inplace=True)
expanded_df.drop(columns=['Key-role'], errors='ignore', inplace=True)

expanded_df.rename(columns={'keyrole': 'Key Role'}, inplace=True)

# # Display the first few rows of the DataFrame to see the new 'keyrole' column
# print(expanded_df.head())


In [8]:
# Group by 'Player Name' and 'Team', then aggregate 'Match ID' into a list in expanded_df
# player_matches = expanded_df.groupby(['Player Name', 'Team'])['Match ID'].apply(list).reset_index()
player_matches = expanded_df.groupby(['Player Name', 'Team']).agg({
    'Match ID': list,
    'Season': list,
    'Teams': list,
    'Key Role': list,
    'Contribution Description': list,
    'Outcome': list,
    'Impact Rating': list
}).reset_index()
# Ensure 'Team' column exists in player_names_df by adding it
player_names_df['Team'] = player_names_df['Player Name'].map(player_team_mapping)


# Merge this information with player_names_df
final_player_df = pd.merge(player_names_df, player_matches, on=['Player Name', 'Team'], how='left')


# Initialize an empty DataFrame to store the result
# output_df = pd.DataFrame(columns=['Player Name', 'Team', 'Match ID'])
output_df = pd.DataFrame(columns=['Player Name', 'Team', 'Match ID','Season','Teams','Key Role','Contribution Description','Outcome','Impact Rating'])

# Iterate over each row of the merged DataFrame
for index, row in final_player_df.iterrows():
    # Get the player name, team, and match IDs
    player_name = row['Player Name']
    team = row['Team']
    match_ids = row['Match ID']
    seasons = row['Season']
    opponents = row['Teams']
    keyrole = row['Key Role']
    contributions = row['Contribution Description']
    outcomes = row['Outcome']
    impact_rating = row['Impact Rating']
    
# Create the header row
    header_row = {
        'Player Name': player_name,
        'Team': team,
        'Match ID': match_ids[0],
        'Season': seasons[0],
        'Teams': opponents[0],
        'Key Role': keyrole[0],
        'Contribution Description': contributions[0],
        'Outcome': outcomes[0],
        'Impact Rating': impact_rating[0]
    }
    output_df = output_df.append(header_row, ignore_index=True)
# Create subsequent rows for match IDs
    for i in range(1, len(match_ids)):
        match_row = {
            'Player Name': '',
            'Team': '',
            'Match ID': match_ids[i],
            'Season': seasons[i],
            'Teams': opponents[i],
            'Key Role': keyrole[i],
            'Contribution Description': contributions[i],
            'Outcome': outcomes[i],
            'Impact Rating': impact_rating[i]
        }
        output_df = output_df.append(match_row, ignore_index=True)

# # Display the resulting DataFrame
# print(output_df.to_string(index = False))
# Convert DataFrame to HTML
html_output = output_df.to_html(index = False)

# Display HTML table
display(HTML(html_output))

# Optional: Save to csv
output_df.to_csv('output.csv', index=False)

C:\Users\nazar\AppData\Local\Temp\ipykernel_16012\3150864548.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(header_row, ignore_index=True)
C:\Users\nazar\AppData\Local\Temp\ipykernel_16012\3150864548.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(match_row, ignore_index=True)
C:\Users\nazar\AppData\Local\Temp\ipykernel_16012\3150864548.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(match_row, ignore_index=True)
C:\Users\nazar\AppData\Local\Temp\ipykernel_16012\3150864548.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df 

C:\Users\nazar\AppData\Local\Temp\ipykernel_16012\3150864548.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(header_row, ignore_index=True)
C:\Users\nazar\AppData\Local\Temp\ipykernel_16012\3150864548.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(header_row, ignore_index=True)
C:\Users\nazar\AppData\Local\Temp\ipykernel_16012\3150864548.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(match_row, ignore_index=True)
C:\Users\nazar\AppData\Local\Temp\ipykernel_16012\3150864548.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df

C:\Users\nazar\AppData\Local\Temp\ipykernel_16012\3150864548.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(header_row, ignore_index=True)
C:\Users\nazar\AppData\Local\Temp\ipykernel_16012\3150864548.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(header_row, ignore_index=True)
C:\Users\nazar\AppData\Local\Temp\ipykernel_16012\3150864548.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(header_row, ignore_index=True)
C:\Users\nazar\AppData\Local\Temp\ipykernel_16012\3150864548.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_d

Player Name,Team,Match ID,Season,Teams,Key Role,Contribution Description,Outcome,Impact Rating
Faf du Plessis,RCB,1,2021,CSK vs KKR,Batsman,"Faf du Plessis led the charge, making 86 as he anchored half-century stands for each of the three wickets that fell during the course of Super Kings' 192.",CSK won by 27 runs,81.91
,,33,2023,SRH vs RCB,Batsman,"Du Plessis had key partnership with Kohli, he scored 71",RCB won by 8 wickets (with 4 balls remaining),71.80
,,39,2023,PBKS vs RCB,Batsman,"He hit 84 off 56 balls on a sluggish Mohali pitch to haul Royal Challengers Bangalore to 174 for 4, despite a late slowdown",RCB won by 24 runs,102.05
Shardul Thakur,CSK,1,2021,CSK vs KKR,Bowler,Shardul Thakur took the most wickets along with other bowlers which led to their victory.,CSK won by 27 runs,57.80
Sunil Narine,KKR,2,2021,RCB vs KKR,Batsman,"Three balls of Sunil Narine, with the bat, changed the game, putting Kolkata Knight Riders one step closer to the IPL final.",KKR won by 4 wickets (with 2 balls remaining),158.09
Virat Kohli,RCB,2,2021,RCB vs KKR,Batsman,Pressure was mounting on Kohli. He had slowed down once again after the powerplay - 24 off 16 and 15 off 17 - and his desperation was apparent.,KKR won by 4 wickets (with 2 balls remaining),46.38
,,3,2021,RCB vs MI,Batsman,Virat Kohli struck another half-century,RCB won by 54 runs,62.36
,,17,2024,SRH vs RCB,Batsman,"Virat Kohli also scored his 53rd IPL fifty, but his 51 in 43 balls came at a strike rate of 118.60, following a big slowdown.",RCB won by 35 runs,30.11
,,33,2023,SRH vs RCB,Batsman,Kohli scored a century,RCB won by 8 wickets (with 4 balls remaining),99.06
Harshal Patel,PBKS,3,2021,RCB vs MI,Batsman,Glenn Maxwell provided middle-overs firepower and key wickets,RCB won by 54 runs,34.83
